## ExMAS
> Exact Matching of Attractive Shared rides (ExMAS) for system-wide strategic evaluations
> 
> Kucharski R., Cats O. ,_Transportation Research Part: B_, 2020

---

In [1]:
import os
cwd = os.getcwd()

In [2]:
os.chdir(os.path.join(cwd,'../..'))
import ExMAS.main
import ExMAS.utils

In [3]:
from ExMAS.utils import inData as inData
from ExMAS.main import matching

In [4]:
import random
%load_ext autoreload
%autoreload 2

### prepare

1. __load the default config__ (see more details in `ExMAS/notebooks/01 configuration.ipynb`)

In [5]:
params = ExMAS.utils.get_config('ExMAS/data/configs/default.json') # load the default 
params.nP = 1000
params.simTime = 4

In [18]:
params.corona.participation = 0.5
params.corona.p = 0.8
params.corona.initial_share = 0.002

2. __load the network__ (see more details in `ExMAS/notebooks/02 NetworkGraphs.ipynb`)

In [7]:
inData = ExMAS.utils.load_G(inData, params, stats=True)  # download the graph

3. __generate trip requests__ (see more details in `ExMAS/notebooks/02 NetworkGraphs.ipynb`)

In [8]:
inData = ExMAS.utils.generate_demand(inData, params)

1000


In [9]:
params.logger_level = 'INFO'
params.matching_obj = 'u_veh'
params.without_matching = True
params.multi_platform_matching = True

In [10]:
import networkx as nx
import math
import pandas as pd

### run

In [11]:
inData = ExMAS.main(inData, params)

01-09-20 09:18:25-INFO-Initializing pairwise trip shareability between 1000 and 1000 trips.
01-09-20 09:18:25-INFO-creating combinations
01-09-20 09:18:26-INFO-999000	 nR*(nR-1)
01-09-20 09:18:40-INFO-Reduction of feasible pairs by 99.85%
01-09-20 09:18:40-INFO-Degree 2 	Completed
01-09-20 09:18:40-INFO-trips to extend at degree 2 : 4063
01-09-20 09:18:45-INFO-At degree 2 feasible extensions found out of 502 searched
01-09-20 09:18:45-INFO-Degree 3 	Completed
01-09-20 09:18:45-INFO-trips to extend at degree 3 : 502
01-09-20 09:18:45-INFO-At degree 3 feasible extensions found out of 45 searched
01-09-20 09:18:45-INFO-Degree 4 	Completed
01-09-20 09:18:45-INFO-trips to extend at degree 4 : 45
01-09-20 09:18:46-INFO-At degree 4 feasible extensions found out of 5 searched
01-09-20 09:18:46-INFO-Degree 5 	Completed
01-09-20 09:18:46-INFO-trips to extend at degree 5 : 5
01-09-20 09:18:46-INFO-At degree 5 feasible extensions found out of 0 searched
01-09-20 09:18:46-INFO-Degree 6 	Completed
0

In [35]:
def make_population(inData, params):
    """
    generates initial population of _S_uspectible travellers and initial_share if _I_nfected given days prior
    :param params: params.corona.initial_share, params.corona.infected_prior
    :param inData
    :return: inData.population DataFrame with index from inData.requests
    """
    # init population

    share_of_active = params.corona.participation / params.corona.p
    inData.passengers['active'] = False  # part of the simulation
    inData.passengers['state'] = 'S'
    inData.passengers['quarantine_day'] = None
    inData.passengers.active.loc[inData.requests.sample(int(share_of_active * params.nP)).index] = True
    active_ones = inData.passengers[(inData.passengers.active == True)]
    active_ones = active_ones.sample(int(active_ones.shape[0] * params.corona.p))
    active_ones = active_ones[active_ones.state != 'Q']
    inData.passengers['active_today'] = False
    inData.passengers['active_today'].loc[active_ones.index] = True

    inData.passengers['platforms'] = inData.passengers.apply(lambda x: [0] if x.active_today else [-1], axis=1)
    inData.requests['platform'] = inData.requests.apply(lambda row: inData.passengers.loc[row.name].platforms[0],
                                                        axis=1)
    inData.sblts.requests['platform'] = inData.requests['platform']

    # first share infected
    inData.passengers['state'] = inData.passengers.apply(
            lambda x: 'S' if random.random() > params.corona.initial_share else 'I', axis=1)
    inData.passengers['infection_day'] = inData.passengers.apply(
        lambda r: -1 if r.state == "I" else None, axis=1)
    inData.passengers['infected_by'] = None
    return inData

In [37]:
params.corona.participation = 0.625
params.corona.p = 1
inData = make_population(inData,params)
inData.passengers.shape[0], inData.passengers.active.sum(), inData.passengers.active_today.sum() , inData.passengers.groupby('state').size()

(1000,
 625,
 625,
 state
 I      3
 S    997
 dtype: int64)

In [70]:
inData.passengers[(inData.passengers.state == 'I') & (inData.passengers.active_today == True)]

,pos,status,active,state,quarantine_day,active_today,platforms,infection_day,infected_by
98,1433963034,0,True,I,None,True,[0],-1.0,None
333,44768467,0,True,I,None,True,[0],-1.0,None
431,44783329,0,True,I,None,True,[0],-1.0,None


NameError: name 'rides' is not defined

In [1]:
for p in [0.6, 0.8, 0.9]:
    params.p = p
    share_of_active = params.participation / params.p
    inData.passengers['active'] = False
    inData.passengers.active.loc[inData.requests.sample(int(share_of_active*params.nP)).index] = True
    for day in range(1,10):
        print(p, day)
        inData.passengers['active_today'] = inData.passengers.apply(lambda x : is_active_today(x), axis = 1)
        inData.passengers['platforms'] = inData.passengers.apply(lambda x : [0] if x.active_today else [-1], axis = 1)
        inData.requests['platform'] = inData.requests.apply(lambda row: inData.passengers.loc[row.name].platforms[0], axis=1)
        inData.sblts.requests['platform'] = inData.requests['platform']
        inData = matching(inData, params)        
        G = ExMAS.utils.make_graph(inData.sblts.requests[inData.sblts.requests.platform == 0], inData.sblts.schedule, mu = 0.5, plot = False, node_size = 1)
        nx.write_edgelist(G, 'ExMAS/spinoffs/edgelists/matching_p{}_day{}.csv'.format(params.p, day), delimiter = ',', data = False)
        G = ExMAS.utils.make_graph(inData.sblts.requests[inData.sblts.requests.platform == 0], inData.sblts.rides, mu = 0.5, plot = False, node_size = 1)
        nx.write_edgelist(G, 'ExMAS/spinoffs/edgelists/potential_p{}_day{}.csv'.format(params.p, day), delimiter = ',', data = False)

NameError: name 'params' is not defined